In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [3]:
# Load the data
df = pd.read_csv('datasets/dataset_L.csv')

# Separate features and labels
X = df.iloc[:, :-7].values  # First 170 columns as features
y = df.iloc[:, -7:].values  # Last 7 columns as labels

# Normalize the features
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# Convert to tensors
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
y_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, y_tensor)


## CNN with drop out, 3 layers

In [4]:
import torch.nn.functional as F

class NetDropout(nn.Module):
    def __init__(self, n=32):
        super(NetDropout, self).__init__()
        self.n = n
        
        self.conv1 = nn.Conv1d(1, self.n, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout1d(p=0.3)
        self.conv2 = nn.Conv1d(self.n, self.n // 2, kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout1d(p=0.3)
        self.conv3 = nn.Conv1d(self.n // 2, self.n // 2, kernel_size=3, padding=1)
        self.conv3_dropout = nn.Dropout1d(p=0.3)

        # Calculate the correct input size for the fully connected layer
        self.fc1 = nn.Linear((self.n // 2) * (170 // 8), 32)  # 170 // 8 due to three max_pool1d with kernel_size=2
        self.fc2 = nn.Linear(32, 7)
        
        self.lsftmx = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = F.max_pool1d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv3(out)), 2)
        out = self.conv3_dropout(out)

        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = torch.tanh(self.fc1(out))
        out = self.lsftmx(self.fc2(out))
        return out


## CNN with drop out, 4 layers

In [3]:
import torch.nn.functional as F

class NetDropoutBig(nn.Module):
    def __init__(self, n=32):
        super(NetDropoutBig, self).__init__()
        self.n = n
        
        self.conv1 = nn.Conv1d(1, self.n, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout1d(p=0.3)
        self.conv2 = nn.Conv1d(self.n, self.n // 2, kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout1d(p=0.3)
        self.conv3 = nn.Conv1d(self.n // 2, self.n // 2, kernel_size=3, padding=1)
        self.conv3_dropout = nn.Dropout1d(p=0.3)
        self.conv4 = nn.Conv1d(self.n // 2, self.n // 4, kernel_size=3, padding=1)
        self.conv4_dropout = nn.Dropout1d(p=0.3)

        # Calculate the correct input size for the fully connected layer
        self.fc1 = nn.Linear((self.n // 4) * (170 // 16), 32)  # 170 // 16 due to four max_pool1d with kernel_size=2
        self.fc2 = nn.Linear(32, 7)
        
        self.lsftmx = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = F.max_pool1d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv3(out)), 2)
        out = self.conv3_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv4(out)), 2)
        out = self.conv4_dropout(out)

        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = torch.tanh(self.fc1(out))
        out = self.lsftmx(self.fc2(out))
        return out


In [5]:
# 80% training and 20% testing split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
# batch_size = 64
# learning_rate = 0.001
batch_size = 32
learning_rate = 0.0001
n_epochs = 1000

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Initialize the model, loss function, and optimizer
model = NetDropout().to(device)
# model = NetDropoutBig().to(device)
criterion = nn.CrossEntropyLoss()  # Assuming multi-class classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Training loop
for epoch in range(n_epochs):  # Number of epochs
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


C:\IT\anaconda3\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1, Loss: 1.8445943310612538
Epoch 2, Loss: 1.6360719882353127
Epoch 3, Loss: 1.5664033948398026
Epoch 4, Loss: 1.5339192766216376
Epoch 5, Loss: 1.5073629972247944
Epoch 6, Loss: 1.4864669747877457
Epoch 7, Loss: 1.4718741073820574
Epoch 8, Loss: 1.458632116435004
Epoch 9, Loss: 1.4457591772079468
Epoch 10, Loss: 1.4347688692515013
Epoch 11, Loss: 1.4188985853898721
Epoch 12, Loss: 1.4141240750877864
Epoch 13, Loss: 1.403741406612709
Epoch 14, Loss: 1.3983021661883495
Epoch 15, Loss: 1.3862218030442677
Epoch 16, Loss: 1.3817402683460183
Epoch 17, Loss: 1.3725943127756097
Epoch 18, Loss: 1.366009218491771
Epoch 19, Loss: 1.357849935667297
Epoch 20, Loss: 1.3535859801171815
Epoch 21, Loss: 1.3468883182042097
Epoch 22, Loss: 1.3464042877285487
Epoch 23, Loss: 1.3397634887164873
Epoch 24, Loss: 1.3354763323724688
Epoch 25, Loss: 1.3315461633914527
Epoch 26, Loss: 1.3257412916742946
Epoch 27, Loss: 1.319815563653057
Epoch 28, Loss: 1.3166840548258476
Epoch 29, Loss: 1.3128380619809554

In [8]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0
# loader = test_loader
loader = train_loader
results = []
results_x = []
criterion = nn.CrossEntropyLoss() 

with torch.no_grad():  # Disable gradient computation
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU
        outputs = model(inputs)
        results.append(outputs)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Get the predicted class (highest log-probability)
        x, predicted = torch.max(outputs, 1)
        results_x.append((x, predicted))
        
        # print(outputs)
        # Calculate the number of correct predictions
        correct += (predicted == labels.argmax(dim=1)).sum().item()  # labels.argmax(dim=1) for one-hot encoded labels
        total += labels.size(0)

# Calculate average loss and accuracy
avg_test_loss = test_loss / len(loader)
accuracy = correct / total * 100

print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')


Test Loss: 0.9828, Test Accuracy: 62.11%


In [25]:
results_x

[(tensor([-6.5484e-04, -8.1538e-01, -4.7827e-04, -4.9135e-01, -7.4117e-01,
          -4.1964e-01, -8.2496e-01, -6.0348e-01, -1.2153e+00, -9.5400e-01,
          -2.2187e-01, -1.1991e-02, -1.0737e+00, -1.1345e+00, -1.2075e-03,
          -4.2074e-01, -5.6550e-03, -1.4195e+00, -3.9013e-01, -1.0258e+00,
          -6.8798e-01, -8.3966e-01, -8.8595e-01, -1.0810e+00, -8.2226e-01,
          -4.6659e-01, -1.1365e+00, -4.3930e-02, -1.1526e+00, -8.1432e-01,
          -4.7135e-01, -2.0064e-03, -9.6952e-01, -8.3103e-01, -1.0798e-02,
          -5.0635e-01, -9.8217e-01, -6.7467e-01, -9.1014e-01, -1.0901e+00,
          -8.0565e-04, -1.1948e+00, -2.5822e-03, -8.4242e-01, -2.4506e-04,
          -1.7021e-03, -8.8014e-01, -1.0128e+00, -1.2652e+00, -4.0420e-03,
          -1.8199e-03, -5.9337e-01, -8.3482e-01, -1.0430e+00, -9.6519e-01,
          -1.1471e+00, -1.2616e+00, -2.7833e-01, -4.7737e-01, -1.2679e+00,
          -4.3825e-01, -3.9098e-01, -8.5959e-01, -1.3095e+00], device='cuda:0'),
  tensor([4, 2, 2, 

In [14]:
# torch.save(model.state_dict(), "model/model_CNN_L.t")
torch.save(model.state_dict(), "model/model_CNN_L_B.t")

In [49]:
loaded_model = NetDropout().to(device)
loaded_model.load_state_dict(torch.load("model/model_CNN_L.t"))
# loaded_model = NetDropoutBig().to(device)
# loaded_model.load_state_dict(torch.load("model/model_CNN_L_B.t"))

<All keys matched successfully>

# New dataset CNN

## Features model

In [2]:
import torch.nn.functional as F

class CNN_1(nn.Module):
    def __init__(self, n=32):
        super(CNN_1, self).__init__()
        self.n = n
        
        self.conv1 = nn.Conv1d(1, self.n, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout1d(p=0.3)
        self.conv2 = nn.Conv1d(self.n, self.n // 2, kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout1d(p=0.3)
        self.conv3 = nn.Conv1d(self.n // 2, self.n // 2, kernel_size=3, padding=1)
        self.conv3_dropout = nn.Dropout1d(p=0.3)

        # Calculate the correct input size for the fully connected layer
        self.fc1 = nn.Linear((self.n // 2) * (197 // 8), 32)  # 170 // 8 due to three max_pool1d with kernel_size=2
        self.fc2 = nn.Linear(32, 7)
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = F.max_pool1d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv3(out)), 2)
        out = self.conv3_dropout(out)

        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = torch.tanh(self.fc1(out))
        out = self.sigmoid(self.fc2(out))
        return out


In [178]:
# Load the data
df = pd.read_csv('datasets/dataset_2_1.csv')

# Separate features and labels
X = df.iloc[:, :-7].values  # First 198 columns as features
y = df.iloc[:, -7:].values  # Last 7 columns as labels

# Normalize the features
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# Convert to tensors
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
y_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, y_tensor)


In [179]:
# 80% training and 20% testing split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
# batch_size = 64
# learning_rate = 0.001
batch_size = 32
learning_rate = 0.0001
n_epochs = 1000

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Initialize the model, loss function, and optimizer
model = CNN_1().to(device)
# model = NetDropoutBig().to(device)
criterion = nn.BCELoss()  # Assuming multi-class classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [180]:
# Training loop
for epoch in range(n_epochs):  # Number of epochs
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


Epoch 1, Loss: 0.4486325819151742
Epoch 2, Loss: 0.39762668734505063
Epoch 3, Loss: 0.38867109071640743
Epoch 4, Loss: 0.38247274245534624
Epoch 5, Loss: 0.3773257799375625
Epoch 6, Loss: 0.3739976100126902
Epoch 7, Loss: 0.3711932798226674
Epoch 8, Loss: 0.3686487981818971
Epoch 9, Loss: 0.3661191771711622
Epoch 10, Loss: 0.3632771249044509
Epoch 11, Loss: 0.36099889403297786
Epoch 12, Loss: 0.3592999704678853
Epoch 13, Loss: 0.356257537546612
Epoch 14, Loss: 0.3544087184610821
Epoch 15, Loss: 0.3521505890573774
Epoch 16, Loss: 0.34930519700050355
Epoch 17, Loss: 0.34762438671929496
Epoch 18, Loss: 0.3452296314920698
Epoch 19, Loss: 0.3427830933956873
Epoch 20, Loss: 0.34031704664230344
Epoch 21, Loss: 0.33802150658198765
Epoch 22, Loss: 0.3364326817648751
Epoch 23, Loss: 0.3343791537625449
Epoch 24, Loss: 0.3327805301121303
Epoch 25, Loss: 0.3311699575469607
Epoch 26, Loss: 0.32953153269631524
Epoch 27, Loss: 0.3278573333081745
Epoch 28, Loss: 0.3268139088153839
Epoch 29, Loss: 0.324

In [182]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0
# loader = test_loader
loader = train_loader
results = []
results_x = []
criterion = nn.BCELoss() 

with torch.no_grad():  # Disable gradient computation
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU
        outputs = model(inputs)
        results.append(outputs)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Get the predicted class (highest log-probability)
        x, predicted = torch.max(outputs, 1)
        results_x.append((x, predicted))
        
        # print(outputs)
        # Calculate the number of correct predictions
        correct += (predicted == labels.argmax(dim=1)).sum().item()  # labels.argmax(dim=1) for one-hot encoded labels
        total += labels.size(0)

# Calculate average loss and accuracy
avg_test_loss = test_loss / len(loader)
accuracy = correct / total * 100

print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')


Test Loss: 0.2184, Test Accuracy: 64.76%


In [183]:
torch.save(model.state_dict(), "model/model_CNN_2_1.t")

In [ ]:
loaded_model = CNN_1().to(device)
loaded_model.load_state_dict(torch.load("model/model_CNN_2_1.t"))
# loaded_model = NetDropoutBig().to(device)
# loaded_model.load_state_dict(torch.load("model/model_CNN_L_B.t"))

## Mel spectrogram model

In [3]:
import torch
import torch.nn as nn

class CNN_2(nn.Module):
    def __init__(self):
        super(CNN_2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), padding=1)
        self.dropout_conv1 = nn.Dropout2d(p=0.5)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.dropout_conv2 = nn.Dropout2d(p=0.5)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.dropout_conv3 = nn.Dropout2d(p=0.5)
        
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=2, padding=0)
        
        self.fc1 = nn.Linear(128 * 2 * (44 // 2 // 2 // 2), 256)  # Adjust based on the output size from feature extractor
        self.fc1_dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(256, 128)
        self.fc2_dropout = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(128, 7)  # 7 output units for 7 emotions
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.pool(self.dropout_conv1(torch.relu(self.conv1(x))))
        x = self.pool(self.dropout_conv2(torch.relu(self.conv2(x))))
        x = self.pool(self.dropout_conv3(torch.relu(self.conv3(x))))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1_dropout(torch.relu(self.fc1(x)))
        x = self.fc2_dropout(torch.relu(self.fc2(x)))
        x = self.fc3(x)
        return self.sigmoid(x)

In [185]:
# Load the data
df = pd.read_csv('datasets/dataset_2_2.csv')
N_MELS = 16
TIME_FRAMES = int(704 / N_MELS)

# Separate features and labels
X = df.iloc[:, :-7].values  # First 704 columns as features
y = df.iloc[:, -7:].values  # Last 7 columns as labels

X_reshaped = X.reshape(-1, 1, N_MELS, TIME_FRAMES)

print(X.shape, X_reshaped.shape, X_reshaped[0].shape)
# Normalize the features
# scaler = StandardScaler()
# X = scaler.fit_transform(X_reshaped)

# Convert to tensors
X_tensor = torch.tensor(X_reshaped, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, y_tensor)


(20990, 704) (20990, 1, 16, 44) (1, 16, 44)


In [186]:
# 80% training and 20% testing split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
# batch_size = 64
# learning_rate = 0.001
batch_size = 32
learning_rate = 0.0001
n_epochs = 500

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Initialize the model, loss function, and optimizer
model = CNN_2().to(device)
# model = NetDropoutBig().to(device)
criterion = nn.BCELoss()  # Assuming multi-class classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [187]:
# Training loop
for epoch in range(n_epochs):  # Number of epochs
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


Epoch 1, Loss: 0.4791552142869859
Epoch 2, Loss: 0.43286370549883163
Epoch 3, Loss: 0.4252861285777319
Epoch 4, Loss: 0.42062427009854997
Epoch 5, Loss: 0.4180423634960538
Epoch 6, Loss: 0.41535464519546145
Epoch 7, Loss: 0.413453475804556
Epoch 8, Loss: 0.4105277957802727
Epoch 9, Loss: 0.40609642721357797
Epoch 10, Loss: 0.4004548645019531
Epoch 11, Loss: 0.3944721519379389
Epoch 12, Loss: 0.3890017412390028
Epoch 13, Loss: 0.3836770900658199
Epoch 14, Loss: 0.3793657392547244
Epoch 15, Loss: 0.37513214656284877
Epoch 16, Loss: 0.37169519765036446
Epoch 17, Loss: 0.3685377051716759
Epoch 18, Loss: 0.3663645627385094
Epoch 19, Loss: 0.36287262655439834
Epoch 20, Loss: 0.3590447570028759
Epoch 21, Loss: 0.35706651528676353
Epoch 22, Loss: 0.3544048733370645
Epoch 23, Loss: 0.35181389547529673
Epoch 24, Loss: 0.34871350356510705
Epoch 25, Loss: 0.3460703848089491
Epoch 26, Loss: 0.3425153544970921
Epoch 27, Loss: 0.34051265614373344
Epoch 28, Loss: 0.3374666392803192
Epoch 29, Loss: 0.3

In [188]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0
loader = test_loader
# loader = train_loader
results = []
results_x = []
criterion = nn.BCELoss() 

with torch.no_grad():  # Disable gradient computation
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU
        outputs = model(inputs)
        results.append(outputs)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Get the predicted class (highest log-probability)
        x, predicted = torch.max(outputs, 1)
        results_x.append((x, predicted))
        
        # print(outputs)
        # Calculate the number of correct predictions
        correct += (predicted == labels.argmax(dim=1)).sum().item()  # labels.argmax(dim=1) for one-hot encoded labels
        total += labels.size(0)

# Calculate average loss and accuracy
avg_test_loss = test_loss / len(loader)
accuracy = correct / total * 100

print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
# 90% 62%

Test Loss: 0.2210, Test Accuracy: 63.17%


In [189]:
torch.save(model.state_dict(), "model/model_CNN_2_2.t")

In [138]:
loaded_cnn_2_model = CNN_2().to(device)
loaded_cnn_2_model.load_state_dict(torch.load("model/model_CNN_2_2.t"))

<All keys matched successfully>

# Test of combination of models

In [4]:
# Load the data
df_1 = pd.read_csv('datasets/dataset_2_1.csv')
df_2 = pd.read_csv('datasets/dataset_2_2.csv')
N_MELS = 16
TIME_FRAMES = int(704 / N_MELS)

# Separate features and labels
X_1 = df_1.iloc[:, :-7].values  # First 198 columns as features
y_1 = df_1.iloc[:, -7:].values  # Last 7 columns as labels
X_2 = df_2.iloc[:, :-7].values  # First 704 columns as features
y_2 = df_2.iloc[:, -7:].values  # Last 7 columns as labels

X_2_reshaped = X_2.reshape(-1, 1, N_MELS, TIME_FRAMES)
# scaler = StandardScaler()
# X_1 = scaler.fit_transform(X_1)

# Convert to tensors
X_tensor_1 = torch.tensor(X_1, dtype=torch.float32).unsqueeze(1)
y_tensor_1 = torch.tensor(y_1, dtype=torch.float32)
X_tensor_2 = torch.tensor(X_2_reshaped, dtype=torch.float32)
y_tensor_2 = torch.tensor(y_2, dtype=torch.float32)

dataset_1 = TensorDataset(X_tensor_1, y_tensor_1)
dataset_2 = TensorDataset(X_tensor_2, y_tensor_2)


In [6]:
batch_size = 32

test_loader_1 = DataLoader(dataset_1, batch_size=batch_size, shuffle=False)
test_loader_2 = DataLoader(dataset_2, batch_size=batch_size, shuffle=False)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
loaded_cnn_1_model = CNN_1().to(device)
loaded_cnn_1_model.load_state_dict(torch.load("model/model_CNN_2_1.t"))

loaded_cnn_2_model = CNN_2().to(device)
loaded_cnn_2_model.load_state_dict(torch.load("model/model_CNN_2_2.t"))

<All keys matched successfully>

In [8]:
loaded_cnn_1_model.eval()  # Set the model to evaluation mode
loaded_cnn_2_model.eval()  # Set the model to evaluation mode
test_loss1 = 0.0
test_loss2 = 0.0
combined_test_loss = 0.0
correct1 = 0
correct2 = 0
combined_correct = 0
total = 0
results1 = []
results2 = []
results_combined = []
criterion = nn.BCELoss() 

with torch.no_grad():  # Disable gradient computation
    for (data1, data2) in zip(test_loader_1, test_loader_2):
        inputs1, labels1 = data1
        inputs2, labels2 = data2
        
        inputs1, labels1 = inputs1.to(device), labels1.to(device)
        inputs2, labels2 = inputs2.to(device), labels2.to(device)
        
        outputs1 = loaded_cnn_1_model(inputs1)
        outputs2 = loaded_cnn_2_model(inputs2)
        
        combined_outputs = (outputs1 + outputs2) / 2
        results1.append(outputs1)
        results2.append(outputs2)
        results_combined.append(combined_outputs)
        
        # Calculate the loss
        loss1 = criterion(outputs1, labels2)
        loss2 = criterion(outputs2, labels2)
        combined_loss = criterion(combined_outputs, labels1)
        
        test_loss1 += loss1.item()
        test_loss2 += loss2.item()
        combined_test_loss += combined_loss.item()

        # Get the predicted class (highest log-probability)
        x1, predicted1 = torch.max(outputs1, 1)
        x2, predicted2 = torch.max(outputs2, 1)
        _, combined_predicted = torch.max(combined_outputs, 1)
        
        # print(outputs)
        # Calculate the number of correct predictions
        correct1 += (predicted1 == labels1.argmax(dim=1)).sum().item()  # labels.argmax(dim=1) for one-hot encoded labels
        correct2 += (predicted2 == labels2.argmax(dim=1)).sum().item()  # labels.argmax(dim=1) for one-hot encoded labels
        combined_correct += (combined_predicted == labels1.argmax(dim=1)).sum().item()
        
        total += labels1.size(0)

# Calculate average loss and accuracy
avg_test_loss1 = test_loss1 / len(test_loader_1)
avg_test_loss2 = test_loss2 / len(test_loader_2)
avg_combined_test_loss = combined_test_loss / len(test_loader_1)

accuracy1 = correct1 / total * 100
accuracy2 = correct2 / total * 100
combined_accuracy = combined_correct / total * 100

print(f'Test Loss 1: {avg_test_loss1:.4f}, Test Accuracy 1: {accuracy1:.2f}%')
print(f'Test Loss 2: {avg_test_loss2:.4f}, Test Accuracy 2: {accuracy2:.2f}%')
print(f'Combined - Test Loss: {avg_combined_test_loss:.4f}, Test Accuracy: {combined_accuracy:.2f}%')
# 62% 82% 81%

Test Loss 1: 0.2236, Test Accuracy 1: 63.38%
Test Loss 2: 0.1386, Test Accuracy 2: 82.38%
Combined - Test Loss: 0.1669, Test Accuracy: 81.41%
